# Model training for Clothes Recommander 

In [2]:
!pip install kagglehub

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.3 requires pydantic<2.0, but you have pydantic 2.10.6 which is incompatible.
pyasn1-modules 0.2.8 requires pyasn1<0.5.0,>=0.4.6, but you have pyasn1 0.6.1 which is incompatible.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.



  Obtaining dependency information for kagglehub from https://files.pythonhosted.org/packages/96/52/97d3269c429d9215e52c2a7766219c5ad8d53fbb39858c17687874a2524b/kagglehub-0.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for model-signing from https://files.pythonhosted.org/packages/13/86/11fec1355e8f650d647162090b6f113d03493f7b15e65da91635c714fcab/model_signing-0.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for in-toto-attestation from https://files.pythonhosted.org/packages/f7/36/5ff121bd3116f19473965bba8c7f26cee9ade921389317e304c4e216c0f1/in_toto_attestation-0.9.3-py3-none-any.whl.metadata
  Obtaining dependency information for sigstore from https://files.pythonhosted.org/packages/70/f5/324edb6a802438e97e289992a41f81bb7a58a1cda2e49439e7e48896649e/sigstore-3.6.1-py3-none-any.whl.metadata
  Obtaining dependency information for cryptography from https://files.pythonhosted.org/packages/d2/05/5533d30f53f10239616a357f080892026db2d550a40c393d0a8a7af834

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kaborg15/vibrent-clothes-rental-dataset")

print("Path to dataset files:", path)

100%|██████████| 6.17G/6.17G [02:00<00:00, 54.9MB/s]

Extracting files...


Path to dataset files: C:\Users\GLoriaLagrand\.cache\kagglehub\datasets\kaborg15\vibrent-clothes-rental-dataset\versions\7


1. Type de modèle
Tu as raison, un modèle de Computer Vision est idéal. Le pipeline serait :

Feature Extraction : Extraire des caractéristiques de l'image d'entrée (ex: texture, coupe, couleur).
Matching : Comparer ces caractéristiques avec celles d'une base d'images issues des sites de seconde main.
Recommandation : Retourner les produits les plus similaires.

2. Modèles pré-entraînés recommandés
CLIP (Contrastive Language-Image Pretraining - OpenAI) : Permet de comparer texte et images, utile si tu veux matcher un vêtement à une description.
ResNet50 (ImageNet) : Un bon extracteur de features pour classifier les vêtements.
FashionNet / DeepFashion : Spécialisé dans l'analyse et la reconnaissance de vêtements.
Swin Transformer : Plus récent et performant en reconnaissance d’images.
Je te conseille de commencer avec ResNet50 (dispo dans TensorFlow/PyTorch) puis tester CLIP si besoin d’une correspondance plus fine entre image et texte.

In [1]:
pip install tensorflow numpy opencv-python scikit-learn


  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/a4/7d/f1c30a92854540bf789e9cd5dde7ef49bbe63f855b85a2e6b3db8135c591/opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.4/39.5 MB 12.2 MB/s eta 0:00:04
    --------------------------------------- 1.0/39.5 MB 12.2 MB/s eta 0:00:04
   - -------------------------------------- 1.7/39.5 MB 13.3 MB/s eta 0:00:03
   -- ------------------------------------- 2.7/39.5 MB 15.4 MB/s eta 0:00:03
   --- ------------------------------------ 3.8/39.5 MB 17.6 MB/s eta 0:00:03
   ----- ---------------------------------- 5.0/39.5 MB 19.0 MB/s eta 0:00:02
   ------ --------------------------------- 6.0/39.5 MB 19.2 MB/s eta 0:00:02
   ------- -------------------------------- 7.1/39.5 MB 19.6 MB/s eta 0:00:02
   -------- ------------------------------- 8.1/39.5 MB 19.8 MB/s eta 0:00:02
   ---

 1. Charger et préparer le modèle ResNet50

In [2]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image

# Charger le modèle ResNet50 pré-entraîné sur ImageNet, sans la dernière couche de classification
model = ResNet50(weights="imagenet", include_top=False, pooling="avg")

def extract_features(img_path):
    """ Extrait les features d'une image avec ResNet50 """
    img = image.load_img(img_path, target_size=(224, 224))  # Redimensionner l'image
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Normalisation
    
    features = model.predict(img_array)  # Extraction des features
    return features.flatten()  # Aplatir pour une comparaison plus facile


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


 2. Construire une base de données d'images issues de Vinted, eBay, etc.

In [ ]:
import os

# Dossier contenant les images des vêtements de seconde main
database_path = "data/vetements_seconde_main/"  # Modifier avec ton chemin

# Stocker les features de chaque image dans un dictionnaire
database_features = {}

for img_name in os.listdir(database_path):
    img_path = os.path.join(database_path, img_name)
    database_features[img_name] = extract_features(img_path)

print(f"{len(database_features)} images indexées.")


3. Trouver l'image la plus similaire à celle de Zara

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def find_best_match(query_img_path):
    """ Trouve l'image la plus similaire dans la base de données """
    query_features = extract_features(query_img_path)
    
    similarities = {}
    for img_name, features in database_features.items():
        similarity = cosine_similarity([query_features], [features])[0][0]
        similarities[img_name] = similarity
    
    # Trier par similarité décroissante
    best_match = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[0]
    
    return best_match

# Exemple d'utilisation
query_image = "images/zara_robe.jpg"  # Image Zara à rechercher
best_match = find_best_match(query_image)

print(f"L'image la plus similaire est {best_match[0]} avec une similarité de {best_match[1]:.2f}")


 4. Améliorations possibles
✅ Augmenter la base de données : Plus il y a d’images de seconde main, plus les recommandations sont précises.
✅ Utiliser CLIP d'OpenAI : Pour matcher image vs texte si tu veux trouver une "robe rouge Zara" sans image.
✅ Créer une API Flask/FastAPI : Exposer le modèle pour une application web.

